<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
# imports 
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [4]:
# define stop words

STOPWORDS = set(STOPWORDS)
len(STOPWORDS)

337

In [6]:
# define tokenize function using list comprehension

def tokenize(doc):
    
    return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [7]:
# create a list of the reviews to tokenize
reviews = list(yelp['text'])
# create an empty list to store the tokenized reviews
tokens = []
# write a for loop to tokenize and store the reviews
counter = 0 

for review in reviews:
    token = tokenize(reviews[counter])
    tokens.append(token)
    counter = counter + 1

tokens[0]


['beware',
 'fake',
 'fake',
 'fake',
 'small',
 'business',
 'los',
 'alamitos',
 'ca',
 'received',
 'looked',
 'like',
 'legitimate',
 'account',
 'number',
 'called',
 'phone',
 'number',
 'listed',
 'wait',
 'time',
 'hold',
 'said',
 'minutes',
 'leave',
 'message',
 'live',
 'person',
 'phone',
 'matter',
 'number',
 'selected',
 'left',
 'firm',
 'message',
 'contacting',
 'bbb',
 'attorney',
 'company',
 'trying',
 'scam',
 'businesses',
 'illegal']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [11]:
# create a dataframe with tokenized reviews and stars
token_df = pd.DataFrame(data =  {'reviews': yelp['text'], 'token_reviews': tokens, 'stars': yelp['stars']})

In [12]:
token_df.head()

,reviews,token_reviews,stars
0,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...","[beware, fake, fake, fake, small, business, lo...",1
1,Came here for lunch Togo. Service was quick. S...,"[came, lunch, togo, service, quick, staff, fri...",4
2,I've been to Vegas dozens of times and had nev...,"[ve, vegas, dozens, times, stepped, foot, circ...",3
3,We went here on a night where they closed off ...,"[went, night, closed, street, party, best, act...",1
4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...","[stars, bad, price, lunch, seniors, pay, eatin...",4


In [16]:
# import
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

reviews[0]

'BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a legitimate bill for $70 with an account number and all.  I called the phone number listed (866) 273-7934.  The wait time on hold said 20 minutes and to leave a message.  I could not get a live person on the phone no matter what number I selected.  I left a very FIRM message that I would be contacting the BBB and my attorney regarding their company trying to scam businesses. This has to be illegal!!!!!'

In [17]:
# Use Scikit-Learn's CountVectorizer to get word counts for each listing.
vect = CountVectorizer(stop_words = STOPWORDS, max_features=1000)
# fit the model on reviews
vect.fit(reviews)
# create a matrix
dtm = vect.transform(reviews)

dtm = pd.DataFrame(dtm.todense(), columns = vect.get_feature_names())

dtm.head()

,00,10,100,11,12,15,20,24,25,30,...,wow,write,wrong,year,years,yelp,yes,young,yum,yummy
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix
# instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words=STOPWORDS,
                       ngram_range=(1,2),
                       max_df=.97,
                       min_df=3,
                       max_features=1000)

# create a vocabulary and get word counts
dtm = tfidf.fit_transform(reviews)

# get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# view feature matrix as dataframe
dtm.head()

,00,10,10 minutes,100,11,12,15,15 minutes,20,20 minutes,...,write,wrong,year,years,years ago,yelp,yes,young,yum,yummy
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166002,0.211222,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.121427,0.0,0.0,0.0,0.0,0.153053,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.170251,0.152531,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [19]:
# Create a NearestNeighbor Model. 

from sklearn.neighbors import NearestNeighbors

# fit on dtm
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [20]:
# Write a fake review and query for the 10 most similiar reviews, print the text of the reviews
my_review = ['''I absolutely love my kindle! It is the best thing that ever happened to me!''']

In [21]:
# query using nneighbors
# turn the review into a tf idf vector
my_review = tfidf.transform(my_review)

In [23]:
# turn the tf idf vector into a dense matrix
# while plugging it into our knearest neighbors model
nn.kneighbors(my_review.todense())

(array([[1.        , 1.        , 1.07361278, 1.103153  , 1.15348974,
         1.17428988, 1.17742112, 1.18006378, 1.19071625, 1.20242329]]),
 array([[6204, 6311, 2815, 4893, 5044, 3038, 4910, 5207, 8038, 6263]]))

In [26]:
indices = [6204, 6311, 2815, 4893, 5044, 3038, 4910, 5207, 8038, 6263]
counter = 1
for index in indices:
    print(token_df['reviews'][index])
    print('______________________________________________________________________')
    print(counter)
    counter = counter + 1

旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。
______________________________________________________________________
1
天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來
______________________________________________________________________
2
I absolutely love this store so much! Amazing prices and the staff is the friendliest ever. This location is absolutely favorite. Tiffany. Sups amazing girl and I would love to be her best friend!!
______________________________________________________________________
3
Absolutely love this place. They have the best candies and we learned some cool history on Haribo today.  The Haribo cherries and the green frogs are the best. Me and my boyfriend come here whenever we're in the neighborhood. Love the guy in there he has the best personalit

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [27]:
# imports
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV

In [29]:
# define pipeline components
vect = TfidfVectorizer(stop_words = STOPWORDS, ngram_range = (1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [30]:
# define the search space and look for the best hyperparameters of the vectorizer and the classification model
parameters = {
    'vect__max_df': ( 0.75, 1.0), # these are words that are more common
    'vect__min_df': (.02, .05), # these represent words that are rare
    'vect__max_features': (500,2000),
    'clf__n_estimators':(5, 20),
    'clf__max_depth':(15,30)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(token_df['reviews'], token_df['stars'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [32]:
# Prediction on my review
my_good_review = ['''I absolutely love my kindle! It is the best thing that ever happened to me!''']
my_bad_review = ['''This product is awful. Seriously the worst thing I have ever bought EVER!''']
pred = grid_search.predict(my_good_review)
pred2 = grid_search.predict(my_bad_review)
print(pred, pred2)

[5] [1]


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [33]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [34]:
id2word = corpora.Dictionary(token_df['token_reviews'])

Create a bag of words representation of the entire corpus

In [35]:
corpus = [id2word.doc2bow(text) for text in token_df['token_reviews']]

Your LDA model should be ready for estimation: 

In [36]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [37]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()


pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.003055 -0.003032       1        1  10.644785
9     -0.002864 -0.001568       2        1  10.424609
3     -0.002326  0.000183       3        1  10.118363
8      0.002769 -0.001063       4        1  10.029548
6      0.000519 -0.002854       5        1  10.028657
2      0.000241  0.001874       6        1  10.027139
0     -0.001388 -0.002847       7        1   9.926153
1      0.003914  0.003504       8        1   9.842866
7      0.000142  0.003586       9        1   9.660736
5     -0.004062  0.002216      10        1   9.297140, topic_info=           Term         Freq        Total Category  logprob  loglift
45      service  3505.000000  3505.000000  Default  30.0000  30.0000
40         good  4460.000000  4460.000000  Default  29.0000  29.0000
133       place  4560.000000  4560.000000  Default  28.0000  28.0000
18         like  3330.000000  3330.000000  Default  27.0000  27.0000
144        food  4751.000000  4751.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
45      service   219.510284  3505.451904  Topic10  -5.3157  -0.3952
358  definitely   119.921074  1394.630737  Topic10  -5.9203  -0.0781
117        went   120.798851  1433.218994  Topic10  -5.9130  -0.0981
46        staff   118.956299  1505.397461  Topic10  -5.9284  -0.1626
351        come   115.081184  1402.876343  Topic10  -5.9615  -0.1252

[867 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
21960      7  0.618223   abiance
1679       1  0.088085      able
1679       2  0.101039      able
1679       3  0.129537      able
1679       4  0.082904      able
...      ...       ...       ...
24671      6  0.210564      zane
24671      7  0.105282      zane
24671      8  0.105282      zane
24671      9  0.105282      zane
21304      7  0.619017  探したお店でした

[2803 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 4, 9, 7, 3, 1, 2, 8, 6])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Create more visualizations of the LDA results and provide written analysis
* Incorporate RandomizedSearchCV into docoument classification pipeline
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)